In [1]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import scipy.io as sio
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [3]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 64
NUM_EPOCHS = 200

# Architecture
NUM_FEATURES = 28*28
NUM_CLASSES = 2
NUM_CHANNELS = 2

# Other
DEVICE = "cuda:0"
GRAYSCALE = False

In [9]:
mat_contents = sio.loadmat('noma_downlink_2state_data.mat') 
CC = mat_contents['CC']
N = CC.shape[0]
theta = mat_contents['theta'].reshape(N)
x = torch.tensor(CC).float()
y = torch.tensor(theta).long()

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, shuffle=True)
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

In [11]:
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         shuffle=True)
# Checking the dataset
for features, labels in train_loader:  
    print('Image batch dimensions:', features.shape)
    print('Image label dimensions:', labels.shape)
    break

# Checking the dataset
for features, labels in train_loader:  
    print('Image batch dimensions:', features.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 2, 28, 28])
Image label dimensions: torch.Size([64])
Image batch dimensions: torch.Size([64, 2, 28, 28])
Image label dimensions: torch.Size([64])


In [12]:
labels

tensor([0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1])

In [13]:
class LeNet5(nn.Module):

    def __init__(self, num_classes, num_channels):
        super(LeNet5, self).__init__()
        
        in_channels = num_channels
        self.num_classes = num_classes

        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 6*in_channels, kernel_size=5, padding=2),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6*in_channels, 16*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5*in_channels, 120*in_channels),
            nn.Tanh(),
            nn.Linear(120*in_channels, 84*in_channels),
            nn.Tanh(),
            nn.Linear(84*in_channels, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [14]:
torch.manual_seed(RANDOM_SEED)

model = LeNet5(NUM_CLASSES, NUM_CHANNELS)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [15]:
for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

In [16]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [17]:
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/200 | Batch 0000/0063 | Cost: 0.6900
Epoch: 001/200 | Batch 0050/0063 | Cost: 0.6708
Epoch: 001/200 | Train: 76.100%
Time elapsed: 0.10 min
Epoch: 002/200 | Batch 0000/0063 | Cost: 0.6567
Epoch: 002/200 | Batch 0050/0063 | Cost: 0.4597
Epoch: 002/200 | Train: 97.425%
Time elapsed: 0.11 min
Epoch: 003/200 | Batch 0000/0063 | Cost: 0.3689
Epoch: 003/200 | Batch 0050/0063 | Cost: 0.1290
Epoch: 003/200 | Train: 98.775%
Time elapsed: 0.12 min
Epoch: 004/200 | Batch 0000/0063 | Cost: 0.0899
Epoch: 004/200 | Batch 0050/0063 | Cost: 0.0743
Epoch: 004/200 | Train: 98.975%
Time elapsed: 0.12 min
Epoch: 005/200 | Batch 0000/0063 | Cost: 0.0315
Epoch: 005/200 | Batch 0050/0063 | Cost: 0.0172
Epoch: 005/200 | Train: 99.150%
Time elapsed: 0.13 min
Epoch: 006/200 | Batch 0000/0063 | Cost: 0.0468
Epoch: 006/200 | Batch 0050/0063 | Cost: 0.0086
Epoch: 006/200 | Train: 99.625%
Time elapsed: 0.13 min
Epoch: 007/200 | Batch 0000/0063 | Cost: 0.0182
Epoch: 007/200 | Batch 0050/0063 | Cost: 0.016

Epoch: 055/200 | Batch 0050/0063 | Cost: 0.0001
Epoch: 055/200 | Train: 100.000%
Time elapsed: 0.42 min
Epoch: 056/200 | Batch 0000/0063 | Cost: 0.0001
Epoch: 056/200 | Batch 0050/0063 | Cost: 0.0001
Epoch: 056/200 | Train: 100.000%
Time elapsed: 0.43 min
Epoch: 057/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 057/200 | Batch 0050/0063 | Cost: 0.0001
Epoch: 057/200 | Train: 100.000%
Time elapsed: 0.43 min
Epoch: 058/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 058/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 058/200 | Train: 100.000%
Time elapsed: 0.44 min
Epoch: 059/200 | Batch 0000/0063 | Cost: 0.0001
Epoch: 059/200 | Batch 0050/0063 | Cost: 0.0001
Epoch: 059/200 | Train: 100.000%
Time elapsed: 0.44 min
Epoch: 060/200 | Batch 0000/0063 | Cost: 0.0001
Epoch: 060/200 | Batch 0050/0063 | Cost: 0.0002
Epoch: 060/200 | Train: 100.000%
Time elapsed: 0.45 min
Epoch: 061/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 061/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 061/200 | Train: 100.000%
Time el

Epoch: 109/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 109/200 | Train: 100.000%
Time elapsed: 0.74 min
Epoch: 110/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 110/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 110/200 | Train: 100.000%
Time elapsed: 0.74 min
Epoch: 111/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 111/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 111/200 | Train: 100.000%
Time elapsed: 0.75 min
Epoch: 112/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 112/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 112/200 | Train: 100.000%
Time elapsed: 0.75 min
Epoch: 113/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 113/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 113/200 | Train: 100.000%
Time elapsed: 0.76 min
Epoch: 114/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 114/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 114/200 | Train: 100.000%
Time elapsed: 0.76 min
Epoch: 115/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 115/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 115/200 | Train: 100.000%
Time el

Epoch: 163/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 163/200 | Train: 100.000%
Time elapsed: 1.06 min
Epoch: 164/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 164/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 164/200 | Train: 100.000%
Time elapsed: 1.06 min
Epoch: 165/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 165/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 165/200 | Train: 100.000%
Time elapsed: 1.07 min
Epoch: 166/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 166/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 166/200 | Train: 100.000%
Time elapsed: 1.08 min
Epoch: 167/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 167/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 167/200 | Train: 100.000%
Time elapsed: 1.08 min
Epoch: 168/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 168/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 168/200 | Train: 100.000%
Time elapsed: 1.09 min
Epoch: 169/200 | Batch 0000/0063 | Cost: 0.0000
Epoch: 169/200 | Batch 0050/0063 | Cost: 0.0000
Epoch: 169/200 | Train: 100.000%
Time el

In [18]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [19]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.2.weight', 'classifier.2.bias', 'classifier.4.weight', 'classifier.4.bias'])


In [20]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [21]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 99.81%
